In [78]:
import pandas as pd
import numpy as np
import scipy as sp

# Лабораторная работа №3
## Задание 4
## Кондрашин Тимофей 211-365 Вариант 11 (27.04.2023)

### Данные

Для заданного интервального выборочного ряда проверить гипотезу: закон распределения генеральной совокупности является показательным при уровне значимости $\alpha=0.05$

In [79]:
data = pd.DataFrame([154, 99, 42, 34, 20, 7, 4, 1, 4, 1])
n = len(data)
a = 0.05
min_x = 0.5

data.T

,0,1,2,3,4,5,6,7,8,9
0,154,99,42,34,20,7,4,1,4,1


In [80]:
print(f'n = {n}, a = {a}, min_x = {min_x}')

n = 10, a = 0.05, min_x = 0.5


## Шаг 1: Формулировка гипотезы

$H_0$: закон распределения генеральной совокупности является показательным.
$H_1$: закон распределения генеральной совокупности не является показательным.

## Шаг 2: Вычисление характеристик выборки.

### Вычисление среднего значения

$$\overline{x} = \frac{1}{n} \sum_{i=1}^{n} x_i$$

In [81]:
mean = data.mean()
mean

0    36.6
dtype: float64

### Вычисление дисперсии

$$s^2 = \frac{1}{n-1} \sum_{i=1}^{n} (x_i - \overline{x})^2$$

In [82]:
variance = data.var()
variance

0    2613.822222
dtype: float64

### Вычисление стандартного отклонения

$$s = \sqrt{s^2}$$

In [83]:
std = data.std()
std

0    51.125554
dtype: float64

## Шаг 3: Проверка гипотезы

Для проверки гипотезы о нормальности распределения воспользуемся критерием согласия хи-квадрат. Сначала необходимо разбить интервальный ряд на интервалы.

Для этого найдём количество интервалов:

$$k = \lceil \log_2 n + 1 \rceil$$

In [84]:
k = int(np.ceil(np.log2(n) + 1))
k

5

Теперь можно разбить интервальный ряд на интервалы:

$$\Delta = \frac{max(x) - min(x)}{k}$$

In [85]:
delta = (data.max()[0] - data.min()[0]) / k
delta

30.6

Теперь можно построить интервальный ряд:

In [86]:
intervals = []
for i in range(k):
    intervals.append([min_x + i * delta, min_x + (i + 1) * delta])
intervals

[[0.5, 31.1],
 [31.1, 61.7],
 [61.7, 92.30000000000001],
 [92.30000000000001, 122.9],
 [122.9, 153.5]]

Далее, необходимо вычислить выборочные частоты попадания значений в каждый интервал:

$$p_i = \frac{n_i}{n}$$

In [87]:
p = []
for i in range(k):
    p.append(len(data[(data[0] >= min_x + i * delta) & (data[0] < min_x + (i + 1) * delta)]) / n)
p

[0.6, 0.2, 0.0, 0.1, 0.0]

Затем нужно вычислить теоретические (ожидаемые) частоты попадания значений в каждый интервал, если бы распределение было нормальным. Для этого воспользуемся формулой:
$$P_i = \Phi \left( \frac{x_{i+1} - \overline{x}}{s} \right) - \Phi \left( \frac{x_{i} - \overline{x}}{s} \right)$$

In [88]:
P = []
for i in range(k):
    P.append(sp.stats.norm.cdf((intervals[i][1] - mean) / std) - sp.stats.norm.cdf((intervals[i][0] - mean) / std))
P

[array([0.21710363]),
 array([0.23110336]),
 array([0.17375924]),
 array([0.0922668]),
 array([0.03459379])]

In [89]:
intervals_t = pd.DataFrame(intervals, columns=['min', 'max'])
p_t = pd.DataFrame(p, columns=['p (выборочные частоты)'])
P_t = pd.DataFrame(P, columns=['P (ожидаемые частоты)'])
intervals_t.join(p_t).join(P_t).T

,0,1,2,3,4
min,0.500000,31.100000,61.700000,92.300000,122.900000
max,31.100000,61.700000,92.300000,122.900000,153.500000
p (выборочные частоты),0.600000,0.200000,0.000000,0.100000,0.000000
P (ожидаемые частоты),0.217104,0.231103,0.173759,0.092267,0.034594


## Шаг 4: Вычисление значения статистики критерия

Вычислим значение статистики критерия $\chi^2$:

$$\chi^2 = \sum_{i=1}^{k} \frac{(p_i - P_i)^2}{P_i}$$

In [90]:
chi2 = ((np.array(p) - np.array(P)) ** 2 / np.array(P)).sum()
chi2

17.061790654110577

## Шаг 5: Определение критического значения и принятие решения

#### Уровень значимости:
a = 0.05

In [91]:
a

0.05

#### Степени свободы:
$$df = k - 1$$
где
- $k$ - количество интервалов,
- $p$ - количество параметров распределения, которые были оценены по выборке.

In [92]:
df = k - 1
df

4

Таблица распределения хи-квадрат ($\chi^2$) со степенями свободы 4 и уровнем значимости 0.05 даёт критическое значение $\chi^2 = 9.488$.

Так как вычисленное значение статистики критерия Пирсона ($\chi^2 = 17.061790654110577$) превышает критическое значение ($\chi^2 = 9.488$), то мы отвергаем нулевую гипотезу $H_0$ и принимаем альтернативную гипотезу $H_1$. Это означает, что выборка получена из распределения, отличного от показательного.


## Шаг 5: Вывод

Таким образом, на основе данных выборки мы можем с уверенностью утверждать, что выборка получена из распределения, отличного от показательного.